Nessie Spark SQL Demo with NBA Dataset
============================
This demo showcases how to use Nessie Python API along with Spark3 from Iceberg

Initialize Pyspark + Nessie environment
----------------------------------------------

In [1]:
import os
import findspark
from pyspark.sql import *
from pyspark import SparkConf, SparkContext
from py4j.java_gateway import java_import
findspark.init()

conf = SparkConf()
conf.set("spark.jars.packages", "org.apache.iceberg:iceberg-spark3-runtime:0.11.1")
conf.set("spark.sql.execution.pyarrow.enabled", "true")
conf.set("spark.sql.catalog.nessie.warehouse", 'file://' + os.getcwd() + '/spark_warehouse')
conf.set("spark.sql.catalog.nessie.url", "http://localhost:19120/api/v1")
conf.set("spark.sql.catalog.nessie.ref", "main")
conf.set("spark.sql.catalog.nessie.catalog-impl", "org.apache.iceberg.nessie.NessieCatalog")
conf.set("spark.sql.catalog.nessie.auth_type", "NONE")
conf.set("spark.sql.catalog.nessie.cache-enabled", "false")
conf.set("spark.sql.catalog.nessie", "org.apache.iceberg.spark.SparkCatalog")
conf.set("spark.sql.catalog.spark_catalog", "org.apache.iceberg.spark.SparkSessionCatalog")
spark = SparkSession.builder.config(conf=conf).getOrCreate()
sc = spark.sparkContext
jvm = sc._gateway.jvm

java_import(jvm, "org.apache.iceberg.CatalogUtil")
java_import(jvm, "org.apache.iceberg.catalog.TableIdentifier")
java_import(jvm, "org.apache.iceberg.Schema")
java_import(jvm, "org.apache.iceberg.types.Types")
java_import(jvm, "org.apache.iceberg.PartitionSpec")

Create Extra Spark Contexts
------------------------------

Because of the way Spark Sessions cache Catalogs we can only have 1 Nessie Catalog per Spark Session. Below we create a few Spark Sessions to deal with the different Nessie branches we will use in the course of the Demo. Typically in a real life scenario we would create one Nessie catalog per Spark insance and each of the below sections would be run by different users on different branches.

In [2]:
# session for dev branch
spark_dev = spark.newSession()
spark_dev.conf.set("spark.sql.catalog.nessie.ref", "dev" )

# session for ETL branch
spark_etl = spark.newSession()
spark_etl.conf.set("spark.sql.catalog.nessie.ref", "etl" )

# session for experiment branch
spark_experiment = spark.newSession()
spark_experiment.conf.set("spark.sql.catalog.nessie.ref", "experiment" )

Set up nessie branches
----------------------------

- Branch `main` already exists
- Create branch `dev`
- List all branches (pipe JSON result into jq)

In [3]:
# create a new dev branch
!nessie branch dev

In [4]:
# list all branches
!nessie --verbose branch

* main  5d1e09785da83245caf91d53c8d8c5b753ef21ca comment
  dev   5d1e09785da83245caf91d53c8d8c5b753ef21ca comment



Create tables under dev branch
-------------------------------------

We create two tables under the `dev` branch using the `spark_dev` session:
- `salaries`
- `totals_stats`


In [5]:
# Creating salaries table
spark_dev.sql("CREATE TABLE IF NOT EXISTS nessie.nba.salaries (Season STRING, Team STRING, Salary STRING, Player STRING) USING iceberg")
salaries_df = spark_dev.read.csv("data/nba-dataset/salaries.csv", header=True)
salaries_df.write.format("iceberg").mode("overwrite").save("nessie.nba.salaries")

# Creating totals_stats table
spark_dev.sql("CREATE TABLE IF NOT EXISTS nessie.nba.totals_stats (Season STRING, Age STRING, Team STRING, ORB STRING, DRB STRING, TRB STRING, AST STRING, STL STRING, BLK STRING, TOV STRING, PTS STRING, Player STRING, RSorPO STRING) USING iceberg")
totals_stats_df = spark_dev.read.csv("data/nba-dataset/totals_stats.csv", header=True)
totals_stats_df.write.format("iceberg").mode("overwrite").save("nessie.nba.totals_stats")


In [6]:
# notice how we view the data of the salaries table on the dev branch via @dev
spark.sql("select * from nessie.nba.`salaries@dev`").toPandas()

,Season,Team,Salary,Player
0,2003-04,Cleveland Cavaliers,$4018920,Lebron James
1,2004-05,Cleveland Cavaliers,$4320360,Lebron James
2,2005-06,Cleveland Cavaliers,$4621800,Lebron James
3,2006-07,Cleveland Cavaliers,$5828090,Lebron James
4,2007-08,Cleveland Cavaliers,$13041250,Lebron James
5,2008-09,Cleveland Cavaliers,$14410581,Lebron James
6,2009-10,Cleveland Cavaliers,$15779912,Lebron James
7,2010-11,Miami Heat,$14500000,Lebron James
8,2011-12,Miami Heat,$16022500,Lebron James
9,2012-13,Miami Heat,$17545000,Lebron James


Check generated tables
----------------------------

Check tables generated under the `dev` branch (and that the `main` branch does not have any tables)

In [7]:
# there are no tables on the main branch
!nessie contents --list

In [8]:
# we should see the salaries & totals_stats tables on the dev branch
!nessie contents --list --ref dev

ICEBERG_TABLE:
	nba.salaries
	nba.totals_stats



Note that the `dev` and `main` branches point to different commits now

In [9]:
# list all branches
!nessie --verbose branch

* main  5d1e09785da83245caf91d53c8d8c5b753ef21ca comment
  dev   f1b50499f018c7aac160682579b34b3ffe72d65c comment



Dev promotion
-------------

Promote `dev` branch to `main`.

* `main` now has the same tables as `dev`
* `main` and `dev` point to the same commit

In [10]:
# merge dev into main
!nessie merge dev -b main --force

In [11]:
# list all branches
!nessie --verbose branch

* main  f1b50499f018c7aac160682579b34b3ffe72d65c comment
  dev   f1b50499f018c7aac160682579b34b3ffe72d65c comment



Create `etl` branch
----------------------

- Create a branch `etl` out of `main`
- add data to `salaries`
- alter the schema of `totals_stats`
- create table `allstar_games_stats`
- query the tables in `etl`
- query the tables in `main`
- promote `etl` branch to `main`

In [12]:
# create the etl branch based on main
!nessie branch etl main

In [13]:
# add some salaries for Kevin Durant
Salary = Row("Season", "Team", "Salary", "Player")
kevin_durant = spark_etl.createDataFrame([
    Salary("2017-18", "Golden State Warriors", "$25000000", "Kevin Durant"),
    Salary("2018-19", "Golden State Warriors", "$30000000", "Kevin Durant"),
    Salary("2019-20", "Brooklyn Nets", "$37199000", "Kevin Durant"),
    Salary("2020-21", "Brooklyn Nets", "$39058950", "Kevin Durant")])
kevin_durant.write.format("iceberg").mode("append").save("nessie.nba.salaries")

In [14]:
# dropping a column in the totals_stats table
spark_etl.sql("ALTER TABLE nessie.nba.totals_stats DROP COLUMN Age")

DataFrame[]

In [15]:
# Creating allstar_games_stats table and viewing the contents
spark_etl.sql("CREATE TABLE IF NOT EXISTS nessie.nba.allstar_games_stats (Season STRING, Age STRING, Team STRING, ORB STRING, TRB STRING, AST STRING, STL STRING, BLK STRING, TOV STRING, PF STRING, PTS STRING, Player STRING) USING iceberg")
allstar_games_stats_df = spark_etl.read.csv("data/nba-dataset/allstar_games_stats.csv", header=True)
allstar_games_stats_df.write.format("iceberg").mode("overwrite").save("nessie.nba.allstar_games_stats")

spark.sql("select * from nessie.nba.`allstar_games_stats@etl`").toPandas()

,Season,Age,Team,ORB,TRB,AST,STL,BLK,TOV,PF,PTS,Player
0,2004-05,20,CLE,1,8,6,2,0,3,0,13,Lebron James
1,2005-06,21,CLE,2,6,2,2,0,1,2,29,Lebron James
2,2006-07,22,CLE,0,6,6,1,0,4,0,28,Lebron James
3,2007-08,23,CLE,1,8,9,2,2,4,3,27,Lebron James
4,2008-09,24,CLE,0,5,3,0,0,3,0,20,Lebron James
5,2009-10,25,CLE,1,5,6,4,0,2,1,25,Lebron James
6,2010-11,26,MIA,2,12,10,0,0,4,3,29,Lebron James
7,2011-12,27,MIA,0,6,7,0,0,4,2,36,Lebron James
8,2012-13,28,MIA,0,3,5,1,0,4,0,19,Lebron James
9,2013-14,29,MIA,1,7,7,3,0,1,0,22,Lebron James


In [16]:
# allstar_games_stats is not on the main branch
!nessie contents --list

ICEBERG_TABLE:
	nba.salaries
	nba.totals_stats



In [17]:
# we should see allstar_games_stats on the etl branch
!nessie contents --list --ref etl


ICEBERG_TABLE:
	nba.allstar_games_stats
	nba.salaries
	nba.totals_stats



In [18]:
# now merge the etl branch into main
!nessie merge etl -b main --force

In [19]:
# the etl and main branch should have the same revision
!nessie --verbose branch


* main  ff8ba8e3ec4f3573fa8c5bb67b87d70661f73fad comment
  dev   f1b50499f018c7aac160682579b34b3ffe72d65c comment
  etl   ff8ba8e3ec4f3573fa8c5bb67b87d70661f73fad comment



Create `experiment` branch
--------------------------------

- create `experiment` branch from `main`
- drop `totals_stats` table
- add data to `salaries` table
- compare `experiment` and `main` tables

In [20]:
# create the experiment branch from main
!nessie branch experiment main

In [21]:
# drop the `totals_stats` table
spark_experiment.sql("DROP TABLE IF EXISTS nessie.nba.totals_stats")

DataFrame[]

In [22]:
# add some salaries for Dirk Nowitzki
Salary = Row("Season", "Team", "Salary", "Player")
dirk_nowitzki = spark_experiment.createDataFrame([
    Salary("2015-16", "Dallas Mavericks", "$8333333", "Dirk Nowitzki"),
    Salary("2016-17", "Dallas Mavericks", "$25000000", "Dirk Nowitzki"),
    Salary("2017-28", "Dallas Mavericks", "$5000000", "Dirk Nowitzki"),
    Salary("2018-19", "Dallas Mavericks", "$5000000", "Dirk Nowitzki")])
dirk_nowitzki.write.format("iceberg").mode("append").save("nessie.nba.salaries")

In [23]:
# we should see the salaries and allstar_games_stats tables only
!nessie contents --list --ref experiment

ICEBERG_TABLE:
	nba.allstar_games_stats
	nba.salaries



In [24]:
# main should still see the totals_stats table
!nessie contents --list

ICEBERG_TABLE:
	nba.allstar_games_stats
	nba.salaries
	nba.totals_stats



Let's take a look at the contents of the `salaries` table on the `experiment` branch.
Notice the use of the `nessie` catalog and the use of `@experiment` to view data on the `experiment` branch

In [25]:
spark.sql("select count(*) from nessie.nba.`salaries@experiment`").toPandas()

,count(1)
0,58


and compare to the contents of the `salaries` table on the `main` branch. Notice that we didn't have to specify `@branchName` as it defaulted
to the `main` branch

In [26]:
spark.sql("select count(*) from nessie.nba.salaries").toPandas()

,count(1)
0,54
